In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

import pandas as pd
import matplotlib.pyplot as plt


In [3]:
path = "proccesed_dataset"

In [4]:
# Define path to your dataset (replace with actual path if needed)
def load_and_split_data(directory, validation_split=0.2, test_split=0.1, seed=123):
    # Load the entire dataset
    full_dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        seed=seed,
        shuffle=True,
        label_mode='categorical',
        image_size=(224, 224),
        batch_size=32)  # Adjust batch_size according to your needs

    # Calculate the number of batches needed for each split
    total_batches = len(full_dataset)
    val_batches = int(total_batches * validation_split)
    test_batches = int(total_batches * test_split)
    train_batches = total_batches - val_batches - test_batches

    # Split the dataset into train, validation, and test
    train_dataset = full_dataset.take(train_batches)
    test_dataset = full_dataset.skip(train_batches).take(test_batches)
    validation_dataset = full_dataset.skip(train_batches + test_batches)

    return train_dataset, validation_dataset, test_dataset


In [5]:
base_dir = 'proccesed_dataset'
train_ds, val_ds, test_ds = load_and_split_data(base_dir)

Found 8303 files belonging to 6 classes.


In [ ]:
def make_model(input_shape, num_classes):
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2)
    ])
    
    base_model = MobileNetV2(input_shape=input_shape,
                            include_top=False,
                            weights='imagenet')
    base_model.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(inputs, training=False)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

In [8]:
model = make_model(input_shape=(224, 224, 3), num_classes=6)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
for images, labels in train_ds.take(1):
    print(images.shape)  # Should be (batch_size, 224, 224, 3)
    print(labels.shape)  # Should be (batch_size, 6)

(32, 224, 224, 3)
(32, 6)


In [ ]:
class AccuracyThreshold(Callback):
    def __init__(self, threshold=0.85):
        super(AccuracyThreshold, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get("val_accuracy")
        if val_accuracy is not None:
            if val_accuracy >= self.threshold:
                print(f"\nReached {self.threshold * 100}% accuracy. Stopping training...")
                self.model.stop_training = True

# Instantiate the custom callback with a threshold of 85% (0.85)
accuracy_threshold_callback = AccuracyThreshold(threshold=0.85)

early_stopping_val_acc = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    restore_best_weights=True
)
early_stopping_acc = EarlyStopping(
    monitor='accuracy',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

In [11]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=60,
    callbacks=[accuracy_threshold_callback,early_stopping_val_acc,early_stopping_acc] 
)

Epoch 1/60
182/182 [==============================] - 44s 228ms/step - loss: 11.1024 - accuracy: 0.5067 - val_loss: 9.4562 - val_accuracy: 0.6023
Epoch 2/60
182/182 [==============================] - 40s 222ms/step - loss: 8.2642 - accuracy: 0.6324 - val_loss: 7.2225 - val_accuracy: 0.6478
Epoch 3/60
182/182 [==============================] - 41s 223ms/step - loss: 6.3798 - accuracy: 0.6688 - val_loss: 5.6753 - val_accuracy: 0.6630
Epoch 4/60
182/182 [==============================] - 41s 225ms/step - loss: 5.0640 - accuracy: 0.7014 - val_loss: 4.6078 - val_accuracy: 0.6685
Epoch 5/60
182/182 [==============================] - 40s 222ms/step - loss: 4.1222 - accuracy: 0.7215 - val_loss: 3.8161 - val_accuracy: 0.6837
Epoch 6/60
182/182 [==============================] - 41s 226ms/step - loss: 3.4539 - accuracy: 0.7306 - val_loss: 3.2625 - val_accuracy: 0.6818
Epoch 7/60
182/182 [==============================] - 41s 228ms/step - loss: 2.9439 - accuracy: 0.7426 - val_loss: 2.8319 - val_a

In [12]:
# Assuming test_ds is your test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


26/26 [==============================] - 6s 168ms/step - loss: 1.0955 - accuracy: 0.7356
Test accuracy: 0.7355769276618958
Test loss: 1.0954515933990479


In [13]:
model.save('model')  # Saves to HDF5 file (requires h5py installed)


INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [14]:
import pandas as pd

In [15]:
# Convert to DataFrame
history_df = pd.DataFrame(history.history)

# Save to CSV
history_df.to_csv('model_history.csv', index=False)